Define our prompt

In [2]:
prompt = """
Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not be a list, it should be a JSON object.
${gr.complete_xml_suffix}

an example of output may look like this:
{
	"user_orders": [{                                                                                                   │ │
        "user_id": 1,
        "user_name": "John Mcdonald",
    	"num_orders": 6
    }]
}

"""

Install validators

In [8]:
!guardrails hub install hub://guardrails/valid_length --quiet
!guardrails hub install hub://guardrails/two_words --quiet
!guardrails hub install hub://guardrails/valid_range --quiet
!pip install cohere --quiet

Installing hub://guardrails/valid_length...
✅Successfully installed guardrails/valid_length!


Installing hub://guardrails/two_words...
✅Successfully installed guardrails/two_words!


Installing hub://guardrails/valid_range...
✅Successfully installed guardrails/valid_range!




Defined what our data should look like in pydantic

Note: 1-indexed is an example of passing non-validator instructions on a per field level via the `format` field.
It will throw a validator warning and is to be expected.

In [3]:
from pydantic import BaseModel, Field
from guardrails.hub import ValidLength, TwoWords, ValidRange
from typing import List


class Order(BaseModel):
    user_id: int = Field(description="The user's id.", validators=[("1-indexed", "noop")])
    user_name: str = Field(
        description="The user's first name and last name",
        validators=[TwoWords(on_fail="noop")],
    )

    num_orders: int = Field(
        description="The number of orders the user has placed",
        validators=[ValidRange(0, 50, on_fail="noop")],
    )


class Orders(BaseModel):
    user_orders: List[Order] = Field(
        description="Generate a list of users and how many orders they have placed in the past.",
        validators=[ValidLength(10, 10, on_fail="noop")],
    )

Set up the cohere client. In this notebook, we use an environment variable to pass the cohere API key through

In [5]:
# Add your OPENAI_API_KEY as an environment variable if it's not already set
# import os
# os.environ["COHERE_API_KEY"] = "YOUR_COHERE_API_KEY"

Create the guard and prompt

In [ ]:
from rich import print
import guardrails as gd

guard = gd.Guard.for_pydantic(output_class=Orders)

raw_llm_response, validated_response, *rest = guard(
    messages=[{"role": "user", "content": prompt}],
    model="command-r-08-2024",
    max_tokens=1024,
    temperature=0.3,
)

/Users/dtam/dev/guardrails/guardrails/validator_base.py:590: UserWarning: Validator with id 1-indexed was not found in the registry!  Ignoring...
  warn(f"Validator with id {name} was not found in the registry!  Ignoring...")
/Users/dtam/dev/guardrails/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


By printing the guard's state, we can see the final, validated output, as well as the steps guardrails took to ensure structure.

In [7]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │ │
    │ │ ┃ Role ┃ Content                                                                                      ┃ │ │
    │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │ │
    │ │ │ user │                                                                                              │ │ │
    │ │ │      │ Generate a dataset of fake user orders. Each row of the dataset should be valid. The format  │ │ │
    │ │ │      │ should not be a list, it should be a JSON object.                                            │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ Given below is XML that describes the information to extract from this document and the tags │ │ │
    │ │ │      │ to extract it into.                                                                          │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ <output>                                                                                     │ │ │
    │ │ │      │   <list description="Generate a list of users and how many orders they have placed in the    │ │ │
    │ │ │      │ past." format="guardrails/valid_length: 10 10" name="user_orders" required="true">           │ │ │
    │ │ │      │     <object format="guardrails/valid_length: 10 10" required="true">                         │ │ │
    │ │ │      │       <integer description="The user's id." name="user_id" required="true"></integer>        │ │ │
    │ │ │      │       <string description="The user's first name and last name"                              │ │ │
    │ │ │      │ format="guardrails/two_words" name="user_name" required="true"></string>                     │ │ │
    │ │ │      │       <integer description="The number of orders the user has placed"                        │ │ │
    │ │ │      │ format="guardrails/valid_range: 0 50" name="num_orders" required="true"></integer>           │ │ │
    │ │ │      │     </object>                                                                                │ │ │
    │ │ │      │   </list>                                                                                    │ │ │
    │ │ │      │ </output>                                                                                    │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in  │ │ │
    │ │ │      │ JSON is the `name` attribute of the corresponding XML, and the value is of the type          │ │ │
    │ │ │      │ specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including │ │ │
    │ │ │      │ any types and format requests e.g. requests for lists, objects and specific types. Be        │ │ │
    │ │ │      │ correct and concise. If you are unsure anywhere, enter `null`.                               │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:               │ │ │
    │ │ │      │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`          │ │ │
    │ │ │      │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE',        │ │ │
    │ │ │      │ 'STRING TWO', etc.]}`             